In [84]:
import pandas as pd
from mlbase.db import MLData, Table

In [108]:
class Scorer:
    
    #Собирает кусок запроса sql с условиями скоринга
    def data_to_custom_data(data):
        custom_data = []
        for feature in data:
            name = feature['name']
            sql = 'CASE '
            for score in feature['scores']:
                #Если нижняя граница интервала отсутствует, то проверяем только верхнюю
                if feature['scores'][score]['low'] == None:
                    sql += f'''WHEN 
                {feature['feature']} < {feature['scores'][score]['high']} 
                THEN
                {score}
                '''
                
                #Если верхняя граница интервала отсутствует, проверяем только нижнюю
                elif feature['scores'][score]['high'] == None:
                    sql += f'''WHEN 
                {feature['feature']} >= {feature['scores'][score]['low']} 
                THEN
                {score}
                '''
                #Иначе проверяем обе границы
                else:
                    sql += f'''WHEN 
                {feature['feature']} >= {feature['scores'][score]['low']} 
                AND
                {feature['feature']} < {feature['scores'][score]['high']} 
                THEN
                {score}
                '''
            sql += ''' ELSE 0
                        END'''
            dic = {'name': name,
                  'sql': sql}
            custom_data.append(dic)
        print(custom_data)
        return custom_data
    
    
        
    #Собирает полный запрос sql
    def _custom_scorer_sql(table_name, custom_data, result_column):
        #Создаем и обнуляем колонку с итоговым баллом
        sql = f'''
        alter table {table_name} ADD column if not exists {result_column} float(53) default 0;
        update {table_name} set {result_column} = 0;
        '''
        #Формируем запрос для подсчета скора по каждой фиче
        for feature in custom_data:
            sql += f'''
            alter table {table_name} ADD column if not exists {feature['name']} float(53) default 0;
            update {table_name} set {feature['name']} = 0;
            update {table_name}
            set {feature['name']} = {feature['sql']};
            '''
            #Прибавляем балл по каждой фиче к итоговому баллу
            sql += f'''
            update {table_name}
                set {result_column} = {result_column} + {feature['name']};
                '''

        return sql

    #Получаем на вход data, возвращаем custom_data
    def scorer_sql(table_name, data, result_column):
        custom_data = data_to_custom_data(data)
        return _custom_scorer_sql(table_name, custom_data, result_column)
    
    #Выполняем запрос
    def scorer(table_name, data, result_column, db):
        sql = scorer_sql(table_name, data, result_column)
        db.cur.execute(sql)
        db.commit()
        
    #Если custom_data пользовательский, то пропускается один шаг и запрос выполняется так
    def custom_scorer(table_name, custom_data, result_column, db):
        sql = _custom_scorer_sql(table_name, custom_data, result_column)
        db.cur.execute(sql)
        db.commit()
    
    

In [109]:
#Пример

In [110]:
data = [
    {
        'name': 'score_count',
        'feature': 'count',
        'scores': {
            0: {
                'low': 0,
                'high': 5
            },
            1: {
                'low': 5,
                'high': 10
            },
            2: {
                'low': 10,
                'high': 100
            }
            
        }
    },
    {
        'name': 'score_user',
        'feature': 'user_id',
        'scores': {
            0: {
                'low': None,
                'high': 100
            },
            1: {
                'low': 100,
                'high': 1000
            },
            2: {
                'low': 1000,
                'high': None
            }
            
        }
    }
]

In [111]:
cls = Scorer
cls.scorer('grouped_by_user_title', data, 'result_column', db)

In [ ]:
#Пример для custom_data

In [115]:
custom_data = [{'name': 'score_count',
  'sql': '''CASE 
  WHEN                
  count >= 0                
  AND                
  count < 5                
  THEN 0                
  WHEN                
  count >= 5               
  AND                
  count < 10                
  THEN 1               
  WHEN \n                
  count >= 10
  AND
  count < 100    
  THEN 2
  ELSE 0
  END'''},
 
 {'name': 'score_user',
  'sql': '''CASE 
  WHEN   
  user_id < 100
  THEN 0
  WHEN
  user_id >= 100
  AND
  user_id < 1000
  THEN 1               
  WHEN
  user_id >= 1000   
  THEN 2
  ELSE 0
  END'''}]


In [116]:
cls.custom_scorer('grouped_by_user_title', custom_data, 'result_column', db)

In [ ]:
#Делаем скоринг для Новартиса

In [119]:
custom_data = [
 
 {'name': 'score_detskij_sad__1000',
  'sql': '''CASE 
        WHEN detskij_sad__1000 >= 21 then 2
        WHEN detskij_sad__1000 >= 1 then 1
         ELSE 0
      END '''},
              
  {'name': 'score_avtomobilnaja_parkovka__500',
  'sql': '''CASE 
        WHEN avtomobilnaja_parkovka__500 >= 11 then 2
        WHEN avtomobilnaja_parkovka__500 >= 1 then 1
         ELSE 0
      END '''},
 
 {'name': 'score_all_objects__100',
  'sql': '''CASE 
        WHEN all_objects__100 >= 101 Then 2
        WHEN all_objects__100 >= 1 then 1
         ELSE 0
      END '''}, 
              
  {'name': 'score_routes_1905__100',
  'sql': '''CASE 
        WHEN routes_1905__100 >= 21 then 2
        WHEN routes_1905__100 >= 1 then 1
         ELSE 0
      END  '''},
 
 {'name': 'score_flats',
  'sql': '''CASE 
        WHEN flats__300 >= 3001 then 2
        WHEN flats__300 >= 1 then 1
         ELSE 0
      END '''}
        ]


In [120]:
cls.custom_scorer('drug_stores_base_geo_joined', custom_data, 'total_score', db)

In [ ]:
#Переименуем колонки

In [144]:
def renamer(old_name, new_name):
    db.cur.execute(f'''alter table drug_stores_base_geo_joined
    rename column {old_name} to {new_name}; ''')

In [ ]:
l = [(old_name, new_name) for old_name, new_name in zip(["score_detskij_sad__1000", "score_avtomobilnaja_parkovka__500", "score_all_objects__100", "score_routes_1905__100", "score_flats"], 
                                                  ['"Детский сад, балл"', '"Парковка, балл"', '"Трафик, балл"', '"Дороги, балл"', '"Квартиры, балл"'])]

In [146]:
db.commit()

In [147]:
for old_name, new_name in l:
    print(old_name)
    print(new_name)
    renamer(old_name, new_name)

score_detskij_sad__1000
"Детский сад, балл"
score_avtomobilnaja_parkovka__500
"Парковка, балл"
score_all_objects__100
"Трафик, балл"
score_routes_1905__100
"Дороги, балл"
score_flats
"Квартиры, балл"


In [148]:
db.commit()

In [149]:
db.cur.execute('''alter table drug_stores_base_geo_joined drop column score''')

In [166]:
db.commit()

In [154]:
renamer('total_score', '"Итоговый балл"')

In [156]:
renamer('"Дороги, балл"', '"общественный транспорт, балл"')

In [165]:
db.cur.execute(''' select point_id, "Детский сад, балл", "Парковка, балл", "Трафик, балл", "Квартиры, балл", "общественный транспорт, балл", "Итоговый балл"
into drug_stores_scores
from drug_stores_base_geo_joined;
''')

In [ ]:
db.commit()